# Mining Data to Extract and Visualize Insights in Python

In [1]:
# Load the required libraries
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import prince

# 1. Basic information
Let's load the dataset and extract some basic information

In [2]:
df = pd.read_csv("data.csv")

In [3]:
# Let's take a look at the first values of the dataset
df.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [4]:
# Number of observations and attributes
print(df.shape,"\n")

(11914, 16) 



In [5]:
# Basic statistics and memory usage
print(df.info(), "\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11914 entries, 0 to 11913
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Make               11914 non-null  object 
 1   Model              11914 non-null  object 
 2   Year               11914 non-null  int64  
 3   Engine Fuel Type   11911 non-null  object 
 4   Engine HP          11845 non-null  float64
 5   Engine Cylinders   11884 non-null  float64
 6   Transmission Type  11914 non-null  object 
 7   Driven_Wheels      11914 non-null  object 
 8   Number of Doors    11908 non-null  float64
 9   Market Category    8172 non-null   object 
 10  Vehicle Size       11914 non-null  object 
 11  Vehicle Style      11914 non-null  object 
 12  highway MPG        11914 non-null  int64  
 13  city mpg           11914 non-null  int64  
 14  Popularity         11914 non-null  int64  
 15  MSRP               11914 non-null  int64  
dtypes: float64(3), int64(5

If we take a look at the column Market Category, it has more null values than the other columns.

In [6]:
# Descriptive statistics, excluding NaN values
print(df.describe(), "\n")

               Year    Engine HP  Engine Cylinders  Number of Doors  \
count  11914.000000  11845.00000      11884.000000     11908.000000   
mean    2010.384338    249.38607          5.628829         3.436093   
std        7.579740    109.19187          1.780559         0.881315   
min     1990.000000     55.00000          0.000000         2.000000   
25%     2007.000000    170.00000          4.000000         2.000000   
50%     2015.000000    227.00000          6.000000         4.000000   
75%     2016.000000    300.00000          6.000000         4.000000   
max     2017.000000   1001.00000         16.000000         4.000000   

        highway MPG      city mpg    Popularity          MSRP  
count  11914.000000  11914.000000  11914.000000  1.191400e+04  
mean      26.637485     19.733255   1554.911197  4.059474e+04  
std        8.863001      8.987798   1441.855347  6.010910e+04  
min       12.000000      7.000000      2.000000  2.000000e+03  
25%       22.000000     16.000000    549

In [7]:
# Let's count the unique values of each attribute
for col in df:
    print(col + ' ' + str(df[col].nunique()))

Make 48
Model 915
Year 28
Engine Fuel Type 10
Engine HP 356
Engine Cylinders 9
Transmission Type 5
Driven_Wheels 4
Number of Doors 3
Market Category 71
Vehicle Size 3
Vehicle Style 16
highway MPG 59
city mpg 69
Popularity 48
MSRP 6049


## Cleaning the dataset

Sometimes it's hard to identify when there are spaces before or after a sentence or word. Take a look at the example below, where we add a space at the end of the 1st value at the Transmission Type attribute.

In [8]:
# We print the value, then add a space and printed again to see if we notice any difference
print(df.iloc[0, 6])
df.iloc[0, 6] = df.iloc[0, 6] + ' '
print( df.iloc[0,6])

MANUAL
MANUAL 


In [9]:
# We group by the "Transmission Type" attribute and apply the size() function to count the number of each value.
print(df.groupby(['Transmission Type']).size(), "\n")

Transmission Type
AUTOMATED_MANUAL     626
AUTOMATIC           8266
DIRECT_DRIVE          68
MANUAL              2934
MANUAL                 1
UNKNOWN               19
dtype: int64 



On the results above we notice that there are two different "MANUAL" values, the first counts 2934 and the second just 1.

In [10]:
# Let's order now by the count value
df['Transmission Type'].value_counts()

AUTOMATIC           8266
MANUAL              2934
AUTOMATED_MANUAL     626
DIRECT_DRIVE          68
UNKNOWN               19
MANUAL                 1
Name: Transmission Type, dtype: int64